In [ ]:
pip install tensorflow

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.applications.nasnet import NASNetMobile
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications import EfficientNetB0

In [ ]:
train_dir = "C:/Users/DELL/Desktop/Brain22/BDT/train"
validation_dir = "C:/Users/DELL/Desktop/Brain22/BDT/validation"
test_dir = "C:/Users/DELL/Desktop/Brain22/BDT/test"

In [ ]:
model1 =EfficientNetB0(include_top=False,input_shape=(224, 224, 3), weights='imagenet')

In [ ]:
input_shape = (224,224)

In [ ]:
datagen_train = ImageDataGenerator(rescale=1./255,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  horizontal_flip=True,
                                  vertical_flip=False)


datagen_test = ImageDataGenerator(rescale=1./255)


batch_size = 8
generator_train = datagen_train.flow_from_directory(directory=train_dir,
                                                    target_size=input_shape,
                                                    batch_size=batch_size,
                                                    shuffle=True)

generator_test = datagen_test.flow_from_directory(directory=validation_dir,
                                                  target_size=input_shape,
                                                  batch_size=batch_size,
                                                  shuffle=False)

In [ ]:
def print_layer_trainable():
    for layer in model1.layers:
        print("{0}:\t{1}".format(layer.trainable, layer.name))


print_layer_trainable()

In [ ]:
conv_model = Model(inputs=model1.input, outputs=model1.output)

In [ ]:
from tensorflow.keras.models import Sequential

new_model = Sequential()
 
new_model.add(conv_model)

new_model.add(Flatten())


new_model.add(Dense(1024, activation='relu'))
new_model.add(Dropout(0.5))
new_model.add(Dense(512, activation='relu'))

# Add the final layer for the actual classification.
new_model.add(Dense(2, activation='softmax'))

In [ ]:
optimizer = Adam(lr=1e-5)
loss = 'binary_crossentropy'
metrics = ['categorical_accuracy']
#name="binary_crossentropy"
#metrics=[tf.keras.metrics.BinaryAccuracy(),
new_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss=tf.keras.losses.BinaryCrossentropy(), metrics=metrics)

In [ ]:
epochs = 30
weights = {0:1.5897, 1:1}
steps_per_epoch = generator_train.n / batch_size
steps_test = generator_test.n / batch_size

#steps_per_epoch = 20
#steps_test = 100
history = new_model.fit_generator(generator=generator_train,
                                  epochs=epochs,
                                  steps_per_epoch=steps_per_epoch,
                                  validation_data=generator_test,
                                  validation_steps=steps_test,class_weight = weights)

In [ ]:
new_model.save_weights('C:/Users/DELL/Desktop/Brain22/BDT/Efficient/efficientEpoch30.h5')

In [ ]:
import pandas as pd 
pd.DataFrame(history.history).to_csv("C:/Users/DELL/Desktop/Brain22/BDT/Efficient/efficientEpoch30.csv")

In [ ]:
history = new_model.fit_generator(generator=generator_train,
                                  epochs=5,
                                  steps_per_epoch=steps_per_epoch,
                                  validation_data=generator_test,
                                  validation_steps=steps_test)

In [ ]:
new_model.save_weights('C:/Users/DELL/Desktop/Brain22/BDT/Efficient/efficientEpoch35.h5')
import pandas as pd 
pd.DataFrame(history.history).to_csv("C:/Users/DELL/Desktop/Brain22/BDT/Efficient/efficientEpoch35.csv")

In [ ]:
finaltestCNV_dir = "C:/Users/DELL/Desktop/Brain22/BDT/test"
datagen_finaltestCNV_dir = ImageDataGenerator(rescale=1./255)

generator_finaltestCNV_dir = datagen_finaltestCNV_dir.flow_from_directory(directory=finaltestCNV_dir,
                                                  target_size=input_shape,
                                                  batch_size=batch_size,
                                                  shuffle=False)
steps_finaltestCNV_dir = generator_finaltestCNV_dir.n / batch_size
y_pred = new_model.predict_generator(generator_finaltestCNV_dir,steps = steps_finaltestCNV_dir)

In [ ]:
import pandas as pd 
pd.DataFrame(y_pred).to_csv("C:/Users/DELL/Desktop/Brain22/BDT/Efficient/predEpoch35.csv")

In [ ]:
predicted_class_indices=np.argmax(y_pred,axis=1)
print(predicted_class_indices)

In [ ]:
import pandas as pd 
pd.DataFrame(predicted_class_indices).to_csv("C:/Users/DELL/Desktop/Brain22/BDT/Efficient/predclassEpoch35.csv")

In [ ]:
test_label=np.ones(26)
test_label=test_label.astype(int)
test_label[:10]=0 #Tumor = 1, No-Tumor = 0
print(test_label)

In [ ]:
from sklearn.metrics import confusion_matrix
array = confusion_matrix(test_label, predicted_class_indices)
print(array)
import pandas as pd 

In [ ]:
pd.DataFrame(array).to_csv("C:/Users/DELL/Desktop/Brain22/BDT/Efficient/cmclassEpoch35.csv")

In [ ]:
from sklearn.metrics import classification_report
array = classification_report(test_label, predicted_class_indices,output_dict=True)
print(array)

In [ ]:
import pandas as pd 
pd.DataFrame(array).to_csv("C:/Users/DELL/Desktop/Brain22/BDT/Efficient/reportclassEpoch35.csv")